In [22]:
from allennlp.commands.evaluate import *
from kb.include_all import *
from allennlp.nn import util as nn_util
from allennlp.common.tqdm import Tqdm
import torch

In [3]:
archive_file = "knowbert_wiki_wordnet_model"
cuda_device = 0
line = "banana\tcolor\tyellow"

In [4]:

archive = load_archive(archive_file, cuda_device)

config = archive.config
prepare_environment(config)
reader_params = config.pop('dataset_reader')
print(reader_params)



knowbert_wiki_wordnet_model


In [6]:
reader_params_c = reader_params

In [7]:

if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "food",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))
vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab

duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 0.7944552024205526  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


/home/scruple/miniconda3/envs/knowbert/lib/python3.6/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [8]:
instance = dataset_reader.read_food(line)

['banana', 'color', 'yellow']


In [14]:
print(instance)

Instance with fields:
 	 tokens: TextField of length 6 with text: 
 		[[CLS], banana, has, color, [MASK], [SEP]]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 segment_ids: ArrayField with shape: (6,). 
 	 candidates:  
 	 lm_label_ids: TextField of length 6 with text: 
 		[[CLS], banana, has, color, yellow, [SEP]]
 		and TokenIndexers : {'lm_labels': 'SingleIdTokenIndexer'} 
 	 mask_indicator: ArrayField with shape: (6,). 



In [16]:
instances = dataset_reader.read("../data/food_compact.tsv")

201it [00:00, 3244.78it/s]


In [20]:
instances[0]["candidates"]

In [23]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))

for instance in generator_tqdm:
    batch = nn_util.move_to_device(instance, cuda_device)
    output_dict = model(**batch)
    pooled_output = output_dict.get("pooled_output")
    contextual_embeddings = output_dict.get("contextual_embeddings")
    prediction_scores, seq_relationship_score = model.pretraining_heads(
    contextual_embeddings, pooled_output
    )
    prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
    print("================")
    print(prediction_scores)
    print("================")
    idx = torch.argmin(prediction_scores, dim = 1)
    print(line, vocab[idx])

start


  0%|          | 0/201 [00:00<?, ?it/s]/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-808afw3c/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
  0%|          | 0/201 [01:04<?, ?it/s]

tensor([[[-5.2374, -4.3524, -5.0789,  ..., -4.1331, -4.9955, -1.6057],
         [-6.7754, -5.2830, -6.1453,  ..., -4.6664, -5.9723, -2.9309],
         [-2.2188, -0.8701, -3.2900,  ..., -2.8037, -2.6924,  0.1865],
         ...,
         [-2.1816, -1.0377, -2.0051,  ..., -0.7198, -1.9534,  1.7026],
         [-3.2374, -1.9628, -3.4024,  ..., -1.5293, -2.9011,  1.6084],
         [-4.3510, -3.5991, -4.4943,  ..., -3.4429, -4.7890,  1.0667]]],
       device='cuda:0', grad_fn=<AddBackward0>)


KeyError: tensor([[3, 3, 3,  ..., 3, 3, 3]], device='cuda:0')

In [25]:
prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])

In [27]:
idx = torch.argmin(prediction_scores, dim = 1)
print(line, vocab[idx])

KeyError: tensor([  101, 21966, 25577, 28313,   101, 23681, 25279,  9243],
       device='cuda:0')

In [38]:
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

In [40]:
for i in idx.cpu().numpy():
    print(token2word[i])

[CLS]
##marine
tinted
##tees
[CLS]
##dote
##ritan
##cies
